<a href="https://colab.research.google.com/github/Matthias186/Matthias186.github.io/blob/master/D_Region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages

In [58]:
#pip install pyarrow

# Import Packages

In [59]:
import os
import pandas as pd

from google.colab import drive
from google.cloud import bigquery
from google.oauth2 import service_account

# Mount Drive

In [60]:
#drive.mount('/content/drive')

# Extract

DE.txt --> df_DE

In [61]:
# path = '/content/drive/MyDrive/Data Warehouse/00_Sources/Region/DE.txt'
# df_DE = pd.read_csv(path, header=None, sep='\t',
#                     names= ['country_code',	'postal_code', 	'place_name',
#                             'admin_name1',	'admin_code1',	'admin_name2',
#                             'admin_code2',	'admin_name3',	'admin_code3',
#                             'latitude',	'longitude',	'accuracy'])
# #df_DE

AuszugGV1QAktuell.xlsx --> df_GV

In [62]:
# path = '/content/drive/MyDrive/Data Warehouse/00_Sources/Region/AuszugGV1QAktuell.xlsx'
# df_GV = pd.read_excel(path, sheet_name= 'Onlineprodukt_Gemeinden',header=5,
#                       names=['Satzart', 'Textkennzeichen',	'Land',	'RB',
#                              'Kreis',	'VB',	'Gem',	'Gemeindename',	'Flaeche',
#                              'Bevoelkerung_insgesamt',	'Bevoelkerung_maennlich',
#                              'Bevoelkerung_weiblich',	'Bevoelkerung_je_Flaeche',
#                              'Postleitzahl',	'Laengengrad',	'Breitengrad',
#                              'Reisegebiet_Schluessel',	'Reisegebiet_Bezeichnung',
#                              'Verstaeterung_Schluessel',	'Verstaedterung_Bezeichnung'])
# #df_GV

# Transform

# Load

https://jingsblog.com/2018/11/27/connect-your-jupyter-notebook-to-the-bigquery/

In [63]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/content/drive/MyDrive/Colab Notebooks/DWH/data-warehouse-310713-374eceb69d9b.json'
# Instantiates a client for BigQuery Service
bq = bigquery.Client()

In [65]:
# Query 
QUERY = """
SELECT * FROM `data-warehouse-310713.Testing.Counting`
"""
job = bq.query(QUERY)
df = job.to_dataframe()
#df.tail()

In [66]:
last_value = df['ID'].iloc[-1] + 1
df = df.append({'ID': last_value}, ignore_index=True)

# truncate df 
#df = pd.DataFrame(data={'ID': [1]})

# truncate job
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

In [67]:
# dataset_id = 'data-warehouse-310713:DWH'
# project='data-warehouse-310713'
# dataset = bq.dataset(dataset_id=dataset_id, project=project)

In [68]:
table_id = 'Testing.Counting'

In [69]:
job = bq.load_table_from_dataframe(
    df, table_id, job_config=job_config
)

# Wait for the load job to complete.
job.result()